
## 大模型中，编码器结构和解码器结构的区别？
编码器用于处理理解任务（如分类，语义分析），推理时可一次性处理全部输入，故而训练时，mask是双向建模；

解码器用于处理生成任务（如翻译，文本生成），推理时时逐个token预测，之前预测的token会当作当前的输入，故而训练时，mask时单方向的。




| 特性              | 解码器 (Decoder Layer)       | 编码器 (Encoder Layer)       |
|-------------------|------------------------------|------------------------------|
| 自注意力掩码      | 因果掩码（只能看历史信息）| 无掩码（所有 Token 互相可见） |
| 额外注意力层      | 有（编码器-解码器注意力）| 无                           |
| 训练/推理行为     | 推理时需自回归生成（串行）| 始终并行计算                 |

### 编码器实现

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead)
        self.linear = nn.Linear(d_model, d_model)
        
    def forward(self, x):
        # x: [seq_len, batch_size, d_model]
        # 双向Attention：所有Token相互可见，无Mask
        attn_output, _ = self.self_attn(x, x, x, need_weights=False)
        x = x + attn_output  # 残差连接
        x = self.linear(x)
        return x

# 示例用法
encoder = TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.randn(10, 32, 512)  # [seq_len, batch_size, d_model]
output = encoder(src)  # 一次性处理全部输入

### 解码器结构

In [4]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead)
        self.cross_attn = nn.MultiheadAttention(d_model, nhead)  # 可选（用于Encoder-Decoder结构）
        self.linear = nn.Linear(d_model, d_model)
        
    def forward(self, x, encoder_output=None):
        # x: [tgt_seq_len, batch_size, d_model]
        # 1. 自回归Self-Attention：因果Mask（仅能看到历史信息）
        seq_len = x.size(0)
        causal_mask = torch.triu(torch.ones(seq_len, seq_len) * float('-inf'), diagonal=1).to(x.device)
        self_attn_output, _ = self.self_attn(x, x, x, attn_mask=causal_mask)
        x = x + self_attn_output
        
        # 2. 如果是Encoder-Decoder结构，可加入Cross-Attention（无Mask）
        if encoder_output is not None:
            cross_attn_output, _ = self.cross_attn(x, encoder_output, encoder_output)
            x = x + cross_attn_output
        
        x = self.linear(x)
        return x



# 示例用法（自回归生成）
decoder = TransformerDecoderLayer(d_model=512, nhead=8)
tgt = torch.randn(1, 32, 512)  # 初始输入（如<BOS> Token）
encoder_output = torch.randn(10, 32, 512)  # 假设编码器输出

# 模拟自回归生成（逐个Token预测）
for _ in range(20):  # 生成20个Token
    output = decoder(tgt, encoder_output)
    print(output.shape)
    next_token = output[-1:, :, :]  # 取最后一个Token作为预测结果
    tgt = torch.cat([tgt, next_token], dim=0)  # 将预测结果追加到输入中

torch.Size([1, 32, 512])
torch.Size([2, 32, 512])
torch.Size([3, 32, 512])
torch.Size([4, 32, 512])
torch.Size([5, 32, 512])
torch.Size([6, 32, 512])
torch.Size([7, 32, 512])
torch.Size([8, 32, 512])
torch.Size([9, 32, 512])
torch.Size([10, 32, 512])
torch.Size([11, 32, 512])
torch.Size([12, 32, 512])
torch.Size([13, 32, 512])
torch.Size([14, 32, 512])
torch.Size([15, 32, 512])
torch.Size([16, 32, 512])
torch.Size([17, 32, 512])
torch.Size([18, 32, 512])
torch.Size([19, 32, 512])
torch.Size([20, 32, 512])
